In [ ]:
# --- Colab setup (run this cell first) ---
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies (quiet)
!pip -q install pandas requests openpyxl fuzzywuzzy python-Levenshtein

Mounted at /content/drive


In [ ]:
# ============================================================
# 1) Configuration
# ============================================================

import io
import re
from pathlib import Path
from typing import List, Tuple, Optional, Dict

import pandas as pd
import requests
from fuzzywuzzy import fuzz, process

# ----------------------------
# User settings
# ----------------------------
REBUILD_FROM_SOURCE = False         # True = rebuild caches from Title II URLs / IPEDS file
FUZZY_MATCH_THRESHOLD = 80

# Where to write caches and outputs in Google Drive
PROJECT_DIR = Path("/content/drive/MyDrive/T2_IPEDS_Crosswalk")
RAW_DIR = PROJECT_DIR / "data" / "raw"
CACHE_DIR = PROJECT_DIR / "data" / "cache"
OUTPUT_DIR = PROJECT_DIR / "data" / "outputs"
for d in (RAW_DIR, CACHE_DIR, OUTPUT_DIR):
    d.mkdir(parents=True, exist_ok=True)

TITLEII_CACHE = CACHE_DIR / "titleII_program_universe_2013_2024.csv"
IPEDS_MATCH_CACHE = CACHE_DIR / "ipeds_match_table.csv"

CROSSWALK_OUT = OUTPUT_DIR / "titleII_ipeds_crosswalk.csv"
UNMATCHED_OUT = OUTPUT_DIR / "titleII_unmatched_for_manual_review.csv"

# ----------------------------
# Official Title II source URLs
# ----------------------------
TITLEII_URLS = [
    "https://title2.ed.gov/Public/DataTools/2012/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2013/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2014/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2015/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2016/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2017/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2018/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2019/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2020/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2021/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2022/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2023/AllStates.xlsx",
    "https://title2.ed.gov/Public/DataTools/2024/AllStates.xlsx",
]
TITLEII_SHEET_NAME = "Program"

TARGET_PROGRAM_TYPES = [
    "Traditional",
    "Alternative, not IHE-based",
    "Alternative, IHE-based",
]

# ----------------------------
# State / Territory mapping (Title II full names -> abbreviations)
# ----------------------------
STATE_NAME_TO_ABBR = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA",
    "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO",
    "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH",
    "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT",
    "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS", "Guam": "GU", "Northern Mariana Islands": "MP", "Puerto Rico": "PR",
    "Virgin Islands": "VI",
    "Marshall Islands": "MH", "Micronesia": "FM",
}

# ----------------------------
# IPEDS input file (place your XLSX here)
# ----------------------------
IPEDS_MAIN_FILE = RAW_DIR / "IPEDS_Institution_Reference_Table.xlsx"

# Column names expected in your IPEDS CSV
IPEDS_UNITID_COL = "UnitID"
IPEDS_OFFICIAL_NAME_COL = "Institution Name"
IPEDS_ENTITY_NAME_COL = "Institution (entity) name (HD2023)"
IPEDS_ALIAS_SOURCE_COL = "Institution name alias (HD2023)"
IPEDS_STATE_ABBR_COL = "State abbreviation (HD2023)"

print("Project folders ready:")
print(" - RAW   :", RAW_DIR)
print(" - CACHE :", CACHE_DIR)
print(" - OUTPUT:", OUTPUT_DIR)

Project folders ready:
 - RAW   : /content/drive/MyDrive/T2_IPEDS_Crosswalk/data/raw
 - CACHE : /content/drive/MyDrive/T2_IPEDS_Crosswalk/data/cache
 - OUTPUT: /content/drive/MyDrive/T2_IPEDS_Crosswalk/data/outputs


In [ ]:
# ============================================================
# 2) Canonical name standardization
# ============================================================

def standardize_name(name: str) -> str:
    if pd.isna(name) or name is None:
        return ""
    name = str(name).lower()

    name = name.replace(" at ", "-").replace(" & ", " and ")
    name = re.sub(r"\s*-\s*", "-", name)
    name = re.sub(r"-+", "-", name)

    name = name.replace(" uni ", " university ").replace(" univ ", " university ")
    name = name.replace(" coll ", " college ").replace(" clg ", " college ")
    name = name.replace(" inst ", " institute ")
    name = name.replace(" tech ", " technical ")
    name = name.replace(" st ", " saint ").replace(" mt ", " mount ")
    name = name.replace(" dept ", " department ")
    name = name.replace(" isd ", " independent school district ")
    name = name.replace(" sch ", " school ")
    name = name.replace(" of ", " ")

    if name.startswith("the "):
        name = name[4:]

    name = re.sub(r"[^a-z0-9\s-]", "", name)
    name = re.sub(r"\s+", " ", name)
    name = name.strip().strip("-")
    return name



In [ ]:
# ============================================================
# 3) Build or load Title II program universe
# ============================================================

from typing import List

def download_excel_sheet(
    url: str,
    sheet_name: str = TITLEII_SHEET_NAME,
    timeout: int = 30
) -> pd.DataFrame:
    headers = {"User-Agent": "titleII-ipeds-crosswalk/1.0 (research; colab)"}
    resp = requests.get(url, timeout=timeout, headers=headers)
    resp.raise_for_status()
    return pd.read_excel(
        io.BytesIO(resp.content),
        sheet_name=sheet_name,
        engine="openpyxl"
    )


def build_titleII_program_universe(urls: List[str]) -> pd.DataFrame:
    dfs: List[pd.DataFrame] = []

    for url in urls:
        df = download_excel_sheet(url, sheet_name=TITLEII_SHEET_NAME)

        report_year = int(url.split("/")[-2])
        df["ReportYear"] = report_year

        dfs.append(df)

    # Combine all years
    all_data = pd.concat(dfs, ignore_index=True)

    # Normalize column headers
    all_data.columns = (
        all_data.columns.astype(str)
        .str.replace("\u00A0", " ", regex=False)
        .str.strip()
        .str.replace(" ", "", regex=False)
    )

    # Validate ProgramType
    if "ProgramType" not in all_data.columns:
        raise KeyError("Expected 'ProgramType' column not found in Title II data.")

    # Filter to target program types
    filtered = all_data[
        all_data["ProgramType"].isin(TARGET_PROGRAM_TYPES)
    ].copy()

    # Validate required columns
    required_cols = ["State", "ReportYear", "ProgramCode", "Program", "ProgramType"]
    missing = [c for c in required_cols if c not in filtered.columns]
    if missing:
        raise KeyError(f"Missing required Title II columns: {missing}")

    # Deterministic first appearance (earliest year)
    filtered = filtered.sort_values(
        by=["State", "Program", "ProgramType", "ReportYear"]
    )

    first_instances = (
        filtered
        .drop_duplicates(
            subset=["State", "Program", "ProgramType"],
            keep="first"
        )
        .copy()   # IMPORTANT: prevents SettingWithCopyWarning
    )

    # Standardize program name and state
    first_instances.loc[:, "StandardizedProgram"] = (
        first_instances["Program"].map(standardize_name)
    )

    first_instances.loc[:, "Standardized_State"] = (
        first_instances["State"]
        .astype(str)
        .str.strip()
        .map(STATE_NAME_TO_ABBR)
    )

    # Safety check: unmapped states
    unmapped = (
        first_instances[first_instances["Standardized_State"].isna()]["State"]
        .dropna()
        .unique()
        .tolist()
    )
    if unmapped:
        print("WARNING: Unmapped Title II state/territory values:", unmapped)

    # Final column selection and ordering
    out_cols = [
        "State",
        "Standardized_State",
        "ReportYear",
        "ProgramCode",
        "Program",
        "StandardizedProgram",
        "ProgramType",
    ]

    final_df = (
        first_instances
        .reindex(columns=out_cols)
        .sort_values(
            by=["Standardized_State", "StandardizedProgram", "ProgramType"]
        )
        .reset_index(drop=True)
    )

    return final_df


# ============================================================
# Load from cache or rebuild
# ============================================================

if (not REBUILD_FROM_SOURCE) and TITLEII_CACHE.exists():
    print(f"Loading cached Title II program universe: {TITLEII_CACHE}")
    titleII_universe = pd.read_csv(TITLEII_CACHE)
else:
    print("Rebuilding Title II program universe from official URLs...")
    titleII_universe = build_titleII_program_universe(TITLEII_URLS)
    titleII_universe.to_csv(TITLEII_CACHE, index=False)
    print(f"Saved Title II cache: {TITLEII_CACHE}")

print("Title II universe rows:", len(titleII_universe))
print(titleII_universe.head())


In [ ]:
# ============================================================
# 4) Build or load IPEDS match table (official + entity + aliases)
# ============================================================

def load_ipeds_main(path: Path) -> pd.DataFrame:
    if path.suffix.lower() in [".xlsx", ".xls"]:
        return pd.read_excel(path, dtype=str)
    else:
        return pd.read_csv(path, dtype=str)

    needed = [IPEDS_UNITID_COL, IPEDS_OFFICIAL_NAME_COL, IPEDS_ENTITY_NAME_COL, IPEDS_ALIAS_SOURCE_COL, IPEDS_STATE_ABBR_COL]
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise KeyError(f"IPEDS CSV is missing expected columns: {missing}")
    return df

def build_ipeds_alias_table(df_ipeds_main: pd.DataFrame) -> pd.DataFrame:
    s = df_ipeds_main[[IPEDS_UNITID_COL, IPEDS_ALIAS_SOURCE_COL]].copy()
    s[IPEDS_ALIAS_SOURCE_COL] = (
        s[IPEDS_ALIAS_SOURCE_COL].fillna("").astype(str).replace("nan", "").str.rstrip("|")
    )

    aliases = s.assign(MatchName=s[IPEDS_ALIAS_SOURCE_COL].str.split("|")).explode("MatchName")
    aliases["MatchName"] = aliases["MatchName"].astype(str).str.strip()
    aliases = aliases[aliases["MatchName"] != ""]
    aliases[IPEDS_UNITID_COL] = aliases[IPEDS_UNITID_COL].astype(str)

    aliases["MatchName"] = aliases["MatchName"].map(standardize_name)
    aliases = aliases[aliases["MatchName"] != ""]
    return aliases[[IPEDS_UNITID_COL, "MatchName"]].drop_duplicates().reset_index(drop=True)

def build_ipeds_match_table(df_ipeds_main: pd.DataFrame) -> pd.DataFrame:
    df = df_ipeds_main.copy()
    df[IPEDS_UNITID_COL] = df[IPEDS_UNITID_COL].astype(str)
    df["Standardized_State"] = df[IPEDS_STATE_ABBR_COL].astype(str).str.upper().str.strip()

    official = df[[IPEDS_UNITID_COL, IPEDS_OFFICIAL_NAME_COL, "Standardized_State"]].rename(columns={IPEDS_OFFICIAL_NAME_COL: "MatchName"})
    entity = df[[IPEDS_UNITID_COL, IPEDS_ENTITY_NAME_COL, "Standardized_State"]].rename(columns={IPEDS_ENTITY_NAME_COL: "MatchName"})

    aliases = build_ipeds_alias_table(df)
    aliases = aliases.merge(df[[IPEDS_UNITID_COL, "Standardized_State"]], on=IPEDS_UNITID_COL, how="left")

    out = pd.concat([official, entity, aliases], ignore_index=True)
    out["MatchName"] = out["MatchName"].map(standardize_name)
    out = out[(out["MatchName"] != "") & (out["Standardized_State"] != "")]
    out = out.drop_duplicates(subset=[IPEDS_UNITID_COL, "MatchName", "Standardized_State"]).reset_index(drop=True)
    return out[[IPEDS_UNITID_COL, "MatchName", "Standardized_State"]]

if (not REBUILD_FROM_SOURCE) and IPEDS_MATCH_CACHE.exists():
    print(f"Loading cached IPEDS match table: {IPEDS_MATCH_CACHE}")
    ipeds_match = pd.read_csv(IPEDS_MATCH_CACHE, dtype={IPEDS_UNITID_COL: str})
else:
    print("Building IPEDS match table...")
    df_ipeds_main = load_ipeds_main(IPEDS_MAIN_FILE)
    ipeds_match = build_ipeds_match_table(df_ipeds_main)
    ipeds_match.to_csv(IPEDS_MATCH_CACHE, index=False)
    print(f"Saved IPEDS match cache: {IPEDS_MATCH_CACHE}")

print("IPEDS match rows:", len(ipeds_match))
print(ipeds_match.head())



In [ ]:
# ============================================================
# 5) Matching: Exact within state, then fuzzy within state
# ============================================================

# --- Exact match ---
exact = titleII_universe.merge(
    ipeds_match,
    left_on=["StandardizedProgram", "Standardized_State"],
    right_on=["MatchName", "Standardized_State"],
    how="left"
)

# If multiple matches exist for the same Title II row, keep the first deterministically
exact = exact.sort_values(by=["Standardized_State", "StandardizedProgram", IPEDS_UNITID_COL])
exact = exact.drop_duplicates(subset=["State", "ReportYear", "ProgramCode", "Program", "ProgramType"], keep="first")

exact = exact.rename(columns={IPEDS_UNITID_COL: "Matched_UnitID"})
exact["Match_Method"] = exact["Matched_UnitID"].apply(lambda x: "ExactState" if pd.notna(x) else pd.NA)

unmatched = exact[exact["Matched_UnitID"].isna()].copy()
print("Exact matches:", exact["Matched_UnitID"].notna().sum())
print("Unmatched after exact:", len(unmatched))

# --- Fuzzy match (state-aware) ---
candidates = ipeds_match[["MatchName", "Standardized_State", IPEDS_UNITID_COL]].drop_duplicates().values.tolist()

def find_best_fuzzy_match_state_aware(program_name: str, program_state: str) -> Tuple[Optional[str], Optional[str], Optional[int]]:
    if not program_name or not program_state:
        return None, None, None

    state_choices = [(nm, uid) for (nm, st, uid) in candidates if st == program_state]
    if not state_choices:
        return None, None, None

    names = [x[0] for x in state_choices]
    name_to_unit = {x[0]: x[1] for x in state_choices}

    best = process.extractOne(program_name, names, scorer=fuzz.token_set_ratio)
    if best is None:
        return None, None, None

    matched_name, score = best[0], best[1]
    if score >= FUZZY_MATCH_THRESHOLD:
        return name_to_unit.get(matched_name), matched_name, score
    return None, None, None

fuzzy_rows = []
for _, row in unmatched.iterrows():
    unitid, matched_name, score = find_best_fuzzy_match_state_aware(row["StandardizedProgram"], row["Standardized_State"])
    if unitid:
        fuzzy_rows.append((row["State"], row["ReportYear"], row["ProgramCode"], row["Program"], row["ProgramType"], unitid, matched_name, score))

df_fuzzy = pd.DataFrame(
    fuzzy_rows,
    columns=["State", "ReportYear", "ProgramCode", "Program", "ProgramType", "Matched_UnitID", "Fuzzy_Matched_IPEDS_Name", "Fuzzy_Match_Score"]
)

final = exact.merge(
    df_fuzzy,
    on=["State", "ReportYear", "ProgramCode", "Program", "ProgramType"],
    how="left",
    suffixes=("", "_fuzzy")
)

final["Matched_UnitID"] = final["Matched_UnitID"].fillna(final["Matched_UnitID_fuzzy"])
final.loc[final["Match_Method"].isna() & final["Matched_UnitID"].notna(), "Match_Method"] = "FuzzyState"
final = final.drop(columns=["Matched_UnitID_fuzzy"], errors="ignore")

print("Total matched (exact + fuzzy):", final["Matched_UnitID"].notna().sum())
print("Total unmatched:", final["Matched_UnitID"].isna().sum())



Exact matches: 1836
Unmatched after exact: 1729
Total matched (exact + fuzzy): 2805
Total unmatched: 760


In [ ]:
# ============================================================
# 6) Export outputs
# ============================================================

final.to_csv(CROSSWALK_OUT, index=False)

unmatched_out = final[final["Matched_UnitID"].isna()].copy()
unmatched_out.to_csv(UNMATCHED_OUT, index=False)

print("Wrote outputs:")
print(" - Crosswalk :", CROSSWALK_OUT)
print(" - Unmatched :", UNMATCHED_OUT)

summary = final["Match_Method"].value_counts(dropna=False).to_frame("count")
print("\nMatch method summary:")
print(summary)

